In [1]:
import numpy as np
import pandas as pd
import functions.multi_occupation_network as multi_occupation_network
from  functions.network_figures import bar_plot

# Calibration Excersize 2: One Occupation Per Sector

Suppose there is one type of labor specific to each sector, in other words that $\mathcal{O}=J$. We calibrate our model to mathc BEA input-output tables, and treat the residual, the part of production not accounted for by intermediate inputs, as the labor elasticity. 

In [2]:
data_dir = '../data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_yearly= pd.read_csv(data_dir + 'labor_market_yearly.csv')
dfLabor_market_yearly = dfLabor_market_yearly.sort_values(by=['year', 'BEA_sector'])
dfLabor_market_yearly = dfLabor_market_yearly.dropna(axis=0)
dfLabor_market_yearly = dfLabor_market_yearly[dfLabor_market_yearly['year'] == 2005]
# reformatting parameters
Omega = np.array(dfA.iloc[:, 1:], dtype='float64')
Psi = np.linalg.inv(np.eye(Omega.shape[0])-Omega)
J = Omega.shape[0]
O = J

φ = np.array(dfParam.φ)

epsN = np.diag(np.array(dfParam.α))
epsD = np.array(dfParam.θ_alt).reshape((J,1))

θ = dfLabor_market_yearly['v'] / dfLabor_market_yearly['u'] 
ν = np.ones(J) * 0.5
curlyQ = np.diag(-ν)
curlyF =  np.eye(J) + curlyQ
r = 0.031
curlyT = np.diag(r / (φ*θ**(-ν) - r))
# NOTE: here I am picking the r so the τ's roughly look right. Down the line, we can calibrate τ directly from the data. 
# Also note that tightness is computed from unemployed workers, but in our model it should be computed from the whole work force.
curlyL = np.eye(J)
np.mean(np.diag(curlyT))

0.02282143444747698

## First, we define the shocks we are interested in to allow us to easily change these in the future, that is we define the vectors $d\log\bm{A}$ and $d\log\bm{H}$.

In [3]:
dlog_A = np.zeros((J, 1))
dlog_A[3] = 0.01 
dlog_H = np.zeros((O,1))

We assume Cobb-Douglas production and preferences.

In [4]:
dlog_lam = np.zeros_like(dlog_A)
dlog_epsN = np.zeros_like(epsN)
dlog_epsD = np.zeros_like(epsD)

Finally, we need to make some assumption about how wages change. We begin by assuming that nominal wages do not change in respond to changes in either productivity or the labor force.

In [7]:
#epsW_A = np.zeros((O,J))
gamma_A = 0.95
gamma_H = 1
epsW_A, epsW_H = multi_occupation_network.WageElasticityFunc(gamma_A, gamma_H, Psi, curlyL, epsN)
dlog_wR = multi_occupation_network.WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)

With these pieces in hand, we can now estimate responses to tightness. For details, see the multiple occupations example code.

In [8]:
curlyE = multi_occupation_network.curlyEFunc(dlog_epsN,epsN)
dlog_theta = multi_occupation_network.ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN)
dlog_theta

array([[0.01873109],
       [0.0187259 ],
       [0.0211201 ],
       [0.02274948],
       [0.01841278],
       [0.01755929],
       [0.01851135],
       [0.01825069],
       [0.01882182],
       [0.01954432],
       [0.01959109],
       [0.01897922],
       [0.01843889],
       [0.0188471 ],
       [0.0185996 ],
       [0.01893261],
       [0.01833148]])

In [9]:
num=0
dlog_p = multi_occupation_network.PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, curlyT, curlyL, num=num)
dlog_p

array([[ 0.00000000e+00],
       [ 1.69707961e-05],
       [-7.21586406e-03],
       [-2.06155226e-02],
       [ 3.89352938e-04],
       [ 1.40913368e-03],
       [-3.01725605e-06],
       [ 1.59440494e-04],
       [-1.10982775e-03],
       [-3.04943482e-03],
       [-2.90743447e-03],
       [-1.00832861e-03],
       [ 3.07518385e-04],
       [-1.10525368e-03],
       [ 3.13119201e-04],
       [-6.89375516e-04],
       [ 5.78052086e-04]])

In [10]:
dlog_y = multi_occupation_network.OutputFunc(dlog_A,dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.01116902],
       [0.01115205],
       [0.01838488],
       [0.03178454],
       [0.01077966],
       [0.00975988],
       [0.01117203],
       [0.01100958],
       [0.01227885],
       [0.01421845],
       [0.01407645],
       [0.01217735],
       [0.0108615 ],
       [0.01227427],
       [0.0108559 ],
       [0.01185839],
       [0.01059097]])

We can check labor market clearing holds.

In [11]:
multi_occupation_network.LaborSupply(dlog_H, dlog_theta, curlyF) - multi_occupation_network.LaborDemand(dlog_wR, dlog_y, dlog_p, dlog_epsN, curlyL)

array([[-3.98986399e-17],
       [ 1.56125113e-17],
       [ 0.00000000e+00],
       [ 1.38777878e-17],
       [ 1.73472348e-18],
       [ 3.46944695e-18],
       [-6.93889390e-18],
       [-6.93889390e-18],
       [-3.46944695e-18],
       [ 1.21430643e-17],
       [-2.25514052e-17],
       [-2.60208521e-17],
       [ 0.00000000e+00],
       [-2.08166817e-17],
       [-1.73472348e-17],
       [-3.46944695e-18],
       [-5.20417043e-18]])

Aggregate output changes are.

In [12]:
dlog_aggY = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
dlog_aggY

array([[0.01667662]])

Since labor market frictions are the only source of inefficiency in our network economy, in the absence of labor market frictions our network economy is fully efficient. It is well known that Hulten's theorem holds in efficient network economies with Cobb-Douglas technology. We can therefore easily compare the implications of labor market frictions for aggregate output with the implications of the same technology shocks absent labor market frictions.  

In [13]:
epsD.T @ Psi @ dlog_A

array([[0.00717491]])

In [ ]:
dlog_wR + curlyL @ dlog_p

In [16]:
gamma_A_vec = np.array([0.8,1,1.2])
sectorY_vec = np.zeros((J+1,3))
for i in range(3):
    epsW_A, epsW_H = multi_occupation_network.WageElasticityFunc(gamma_A_vec[i], 1, Psi, curlyL, epsN)
    dlog_wR = multi_occupation_network.WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)
    dlog_theta = multi_occupation_network.ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN)
    dlog_y = multi_occupation_network.OutputFunc(dlog_A,dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
    dlog_aggY = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
    print(gamma_A_vec[i])
    print(dlog_aggY)
    sectorY_vec[:-1, i] = dlog_y.flatten()
    sectorY_vec[-1, i] = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)

0.8
[[0.04518172]]
1.0
[[0.00717491]]
1.2
[[-0.0308319]]


In [ ]:
sector_names = ['Food S', 'Rec', 'Const', 'Dur', 'Educ S', 'Fin', 'Gov', 'Health',
           'Info', 'Mining', 'Nondur', 'Other S', 'Bus S', 'Real Est', 'Retail', 'Trans',
           'Whsale', 'Agg Y']
title = 'Response to 1% Technology Shock in Durables'
xlab = ''
ylab = '$\Delta$ output (pct. points)'
save_path = '../output/figures/durables_shock.png'
labels = ['$\gamma_{A} = 0.95$','$\gamma_{A} = 1$','$\gamma_{A} = 1.05$']
bar_plot(100*sectorY_vec, sector_names, title, xlab, ylab, labels, save_path, rotation=30, fontsize=10, barWidth = 0.25, dpi=300)